Выполните поиск частых наборов объектов в трех различных наборах данных с помощью следующих алгоритмов (или их модификаций): Apriori, FP‑Growth, ECLAT. Наборы данных должны существенно отличаться друг от друга по количеству транзакций и/или типичной длине транзакции (количеству объектов). Варьируйте пороговое значение поддержки (например: 1%, 3%, 5%, 10%, 15%, 20%). Проверьте идентичность результатов, полученных с помощью различных алгоритмов.
Подготовьте список частых наборов, в которых не более семи объектов (разумное количество). Проанализируйте и изложите содержательный смысл полученного результата.
Выполните визуализацию полученных результатов в виде следующих диаграмм: 
- сравнение быстродействия алгоритмов на фиксированном наборе данных при изменяемом пороге поддержки; 
- общее количество частых наборов объектов на фиксированном наборе данных при изменяемом пороге поддержки; 
- максимальная длина частого набора объектов на фиксированном наборе данных при изменяемом пороге поддержки; 
- количество частых наборов объектов различной длины на фиксированном наборе данных при изменяемом пороге поддержки.

Подготовьте отчет о выполнении задания и загрузите его в систему.

In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv('UCI_dataset.csv', header = 0, encoding = "latin")

In [3]:
#data.info()
data.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [4]:
grouped_data = data.groupby('InvoiceNo').agg({'StockCode':','.join}).reset_index()
grouped_data.head(10)

,InvoiceNo,StockCode
0,536365,"85123A,71053,84406B,84029G,84029E,22752,21730"
1,536366,"22633,22632"
2,536367,"84879,22745,22748,22749,22310,84969,22623,2262..."
3,536368,"22960,22913,22912,22914"
4,536369,21756
5,536370,"22728,22727,22726,21724,21883,10002,21791,2103..."
6,536371,22086
7,536372,"22632,22633"
8,536373,"85123A,71053,84406B,20679,37370,21871,21071,21..."
9,536374,21258


In [5]:
"""Заменяем товары на их порядковые номера"""

g_data = grouped_data['StockCode'].str.split(',', expand = True) #разбиение столбцов внутри датафрейма
g_data = g_data.iloc[:,0:20] #взять первые 20 столбцов
val_to_num = pd.value_counts(g_data.values.ravel())
#g_data.index = grouped_data['InvoiceNo']
#g_data.fillna(method = 'ffill',axis = 1, inplace = True)

for i in range(val_to_num.shape[0]):
    val_to_num.iloc[i] = i + 1
print(val_to_num)
for i in range(g_data.shape[1]):
    for j in range(g_data.shape[0]):
        try:
            g_data[i][j] = val_to_num[g_data[i][j]]
        except Exception:
            pass
print(g_data)

22423        1
85123A       2
85099B       3
47566        4
84879        5
          ... 
90165B    3872
17174     3873
35824B    3874
90141E    3875
84966A    3876
Length: 3876, dtype: int64
         0     1     2     3     4     5     6     7     8     9     10    11  \
0         2   340   456   264   242   365   865  None  None  None  None  None   
1       183   266  None  None  None  None  None  None  None  None  None  None   
2         5   358   320   217  1819  1472   675   400    68   108  2902    94   
3        18  1360  1585  1116  None  None  None  None  None  None  None  None   
4       462  None  None  None  None  None  None  None  None  None  None  None   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
25895  3447  None  None  None  None  None  None  None  None  None  None  None   
25896    23   438  None  None  None  None  None  None  None  None  None  None   
25897    63  None  None  None  None  None  None  None  None  None  None  None  

In [6]:
g_data.to_csv("UCI_dataset_norm.csv", header = False, index = False)

2. Нормировка датасета market-basket-optimization.csv

In [7]:
data = pd.read_csv('market-basket-optimization.csv', header = 0, encoding = "latin")
data.head(10)

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,whole milk,butter,yogurt,rice,abrasive cleaner,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,rolls/buns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5,other vegetables,UHT-milk,rolls/buns,bottled beer,liquor (appetizer),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,potted plants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,whole milk,cereals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
g_data = data.iloc[:,1:21]
g_data.columns = [i for i in range(20)]

val_to_num = pd.value_counts(g_data.values.ravel())

for i in range(val_to_num.shape[0]):
    val_to_num.iloc[i] = i + 1
print(val_to_num)
for i in range(g_data.shape[1]):
    for j in range(g_data.shape[0]):
        try:
            g_data[i][j] = val_to_num[g_data[i][j]]
        except Exception:
            pass
print(g_data.head(5))

whole milk                 1
other vegetables           2
rolls/buns                 3
soda                       4
yogurt                     5
                        ... 
kitchen utensil          165
bags                     166
preservation products    167
baby food                168
sound storage medium     169
Length: 169, dtype: int64
   0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0  12   64   22  149  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1   8    5   23  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2   1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3  16    5   33  124  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4   2    1   86   37  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

    15   16   17   18   19  
0  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  
2  NaN  NaN  NaN  NaN  NaN  
3  NaN  NaN  NaN  NaN  NaN  
4  NaN  NaN  NaN  NaN  NaN  


In [9]:
g_data.to_csv("market-basket-optimization_norm.csv", header = False, index = False)

3. Нормировка датасета Instacart Market Basket Analysis

In [10]:
import pandas as pd
data = pd.read_csv('instacart/order_products__train.csv', header = 0, encoding = "latin")
data.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


In [11]:
data['product_id'] = data['product_id'].apply(str)
grouped_data = data.groupby('order_id').agg({'product_id':','.join}).reset_index()
grouped_data.head(10)

,order_id,product_id
0,1,"49302,11109,10246,49683,43633,13176,47209,22035"
1,36,"39612,19660,49235,43086,46620,34497,48679,46979"
2,38,"11913,18159,4461,21616,23622,32433,28842,42625..."
3,96,"20574,30391,40706,25610,27966,24489,39275"
4,98,"8859,19731,43654,13176,4357,37664,34065,35951,..."
5,112,"27104,21174,41860,38273,47209,5876,29217,9047,..."
6,170,"18394,37766,13176,6236,5077,8153,43772,25591,3..."
7,218,"1194,5578,38159,10305,38557"
8,226,"28199,24852,29883,28427,7754,39947,47307,36291..."
9,349,"33000,11361,27695,47672,45633,38015,36968,3083..."


In [12]:
g_data = grouped_data['product_id'].str.split(',', expand = True) #разбиение столбцов внутри датафрейма
g_data = g_data.iloc[:,0:20] #взять первые 20 столбцов
val_to_num = pd.value_counts(g_data.values.ravel())
#g_data.index = grouped_data['InvoiceNo']
#g_data.fillna(method = 'ffill',axis = 1, inplace = True)

for i in range(val_to_num.shape[0]):
    val_to_num.iloc[i] = i + 1

val_to_num = val_to_num.to_dict()
print(val_to_num)
'''

for i in range(g_data.shape[1]):
    for j in range(g_data.shape[0]):
        try:
            g_data[i][j] = val_to_num[g_data[i][j]]
        except Exception:
            pass
print(g_data)
'''

{'24852': 1, '13176': 2, '21137': 3, '21903': 4, '47626': 5, '47766': 6, '47209': 7, '16797': 8, '26209': 9, '27966': 10, '27845': 11, '39275': 12, '30391': 13, '45007': 14, '22935': 15, '24964': 16, '4920': 17, '46979': 18, '4605': 19, '40706': 20, '8518': 21, '42265': 22, '45066': 23, '5876': 24, '31717': 25, '44632': 26, '28204': 27, '43352': 28, '5450': 29, '19057': 30, '8424': 31, '21616': 32, '24184': 33, '30489': 34, '37646': 35, '26604': 36, '49235': 37, '28985': 38, '27104': 39, '44359': 40, '48679': 41, '41950': 42, '27086': 43, '21938': 44, '17794': 45, '49683': 46, '43961': 47, '37067': 48, '12341': 49, '10749': 50, '34126': 51, '5077': 52, '39877': 53, '19660': 54, '9076': 55, '8277': 56, '34969': 57, '35951': 58, '24838': 59, '25890': 60, '35221': 61, '31506': 62, '22035': 63, '46667': 64, '11520': 65, '29487': 66, '8174': 67, '34358': 68, '28842': 69, '20114': 70, '22825': 71, '33198': 72, '39928': 73, '15290': 74, '18465': 75, '27521': 76, '27344': 77, '16759': 78, '387

'\n\nfor i in range(g_data.shape[1]):\n    for j in range(g_data.shape[0]):\n        try:\n            g_data[i][j] = val_to_num[g_data[i][j]]\n        except Exception:\n            pass\nprint(g_data)\n'

In [13]:
print(g_data)

           0      1      2      3      4      5      6      7      8     9   \
0       49302  11109  10246  49683  43633  13176  47209  22035   None  None   
1       39612  19660  49235  43086  46620  34497  48679  46979   None  None   
2       11913  18159   4461  21616  23622  32433  28842  42625  39693  None   
3       20574  30391  40706  25610  27966  24489  39275   None   None  None   
4        8859  19731  43654  13176   4357  37664  34065  35951  43560  9896   
...       ...    ...    ...    ...    ...    ...    ...    ...    ...   ...   
131204  40800  17706  33424  17299  26800  34243   None   None   None  None   
131205   5750   9340  21709  16475  12432   None   None   None   None  None   
131206  15629   4347  34466   6244   6858  30316  35578  32650   None  None   
131207  49235  13565  14233  35548   None   None   None   None   None  None   
131208  35951  16953   4724   None   None   None   None   None   None  None   

           10     11     12     13     14     15   

In [14]:
for i in range(20):
    g_data.iloc[i] = g_data.iloc[i].map(val_to_num)
print(g_data.head(5))

      0     1     2     3     4     5     6    7      8     9     10    11  \
0  14512  1627   139    46  7863     2     7   63    NaN   NaN   NaN   NaN   
1   6703    54    37   265  4993  1715    41   18    NaN   NaN   NaN   NaN   
2   2384  2482   495    32   627   381    69  262   1968   NaN   NaN   NaN   
3    643    13    20  4724    10   115    12  NaN    NaN   NaN   NaN   NaN   
4    148  2157  1394     2  1789  2670  6554   58  13960  6884  1404  4716   

    12    13   14   15    16    17    18   19  
0  NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  
1  NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  
2  NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  
3  NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  
4   10  1220  250  233  7458  3258  2574  370  


In [15]:
g_data.to_csv("order_products__train_norm.csv", header = False, index = False)